In [1]:
import pyspark
from pyspark.sql.functions import col

In [2]:
dataframe = sqlContext.read.csv('data/events.csv')

In [3]:
type(dataframe)

pyspark.sql.dataframe.DataFrame

In [4]:
print(dataframe.columns)

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22']


# Ejercicio 1, evento con mayor frecuencia.

In [5]:
# Versión poco eficiente en memoria ya que el sort se realiza fuera del cluster (creo).
# Esto lo hago para verificar el resultado posterior, para chequear lo que haga de forma Sparkstica.

dataframe.groupBy('_c1').count().sort(col('count').desc()).show()

+-----------------+------+
|              _c1| count|
+-----------------+------+
|   viewed product|528931|
|    brand listing| 98635|
|     visited site| 87378|
|  ad campaign hit| 82827|
|  generic listing| 67534|
|searched products| 56073|
|search engine hit| 50957|
|         checkout| 33735|
|       staticpage|  3598|
|       conversion|  1172|
|             lead|   448|
|            event|     1|
+-----------------+------+



In [6]:
# Vamos a tratar al dataframe como un RDD, y aplicar lo visto en clase de utilizar tuplas para poder contar
# y ordenar los resultados con reduceByKey y un takeOrdered.

dataframe.rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda a,b: a+b).takeOrdered(1, lambda x: -x[1])

[('viewed product', 528931)]

# Ejercicio 2, Top 5 productos vistos.

In [7]:
# Para encontrar el Top 5 de productos, aplicamos la misma idea que en el caso anterior,
# sólo que antes filtramos por tipo de evento para que sean un poco más eficientes las acciones aplicadas.
dataframe.rdd.filter(lambda x: x[1] == 'viewed product')\
    .map(lambda x: (x[5], 1)).reduceByKey(lambda a,b: a+b).takeOrdered(5, lambda x: -x[1])

[('iPhone 6', 47538),
 ('iPhone 5s', 42045),
 ('iPhone 6S', 36930),
 ('iPhone 7', 26330),
 ('Samsung Galaxy S7 Edge', 23062)]